In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [416]:
roll = "091"

In [417]:
import os
import glob
import pandas as pd

In [418]:
DATA_ROOT_PATH = '/content/drive/MyDrive/Dhaka-AI 2020/TestdataHandlabel/'

In [419]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            filename = root.find('filename').text.split('.')[0]
            train_images = []
            train_images.extend(sorted(glob.glob(DATA_ROOT_PATH + '*.jpg')))
            # print(len(train_images))
            for i in train_images:
              x = i.split('/')[-1]
              # print(x)
              idx = x.rfind('.')
              x = x[:idx]
              # print(x)
              if filename in x:
                  filename = x
            value = (filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [420]:

path = os.path.join('/content/drive/MyDrive/Dhaka-AI 2020/Handlabel/1705'+roll,  "*.xml")
all_xml = glob.glob(path)
len(all_xml)

30

In [421]:
# 231.xml has error in parsing as it was not stored in VOC XML format
df = pd.DataFrame()
for i in all_xml:
    try:
        xml_df  = xml_to_csv(i)
        df = df.append(xml_df, ignore_index = True)
    except:
        continue

In [422]:
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,416,416,bus,101,150,174,210
1,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,416,416,rickshaw,6,171,22,216
2,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,416,416,three wheelers (CNG),91,179,104,216
3,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,416,416,rickshaw,163,180,178,219
4,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,416,416,rickshaw,104,172,121,227
...,...,...,...,...,...,...,...,...
284,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,416,416,three wheelers (CNG),319,166,342,179
285,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,416,416,rickshaw,353,167,367,180
286,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,416,416,rickshaw,133,157,179,200
287,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,416,416,three wheelers (CNG),1,152,31,212


In [423]:
df['ImageID'] = df['filename'].map(lambda x: x.rstrip('.jpg')) + ""

In [424]:
df['XMin'] = df['xmin']/df['width']
df['XMax'] = df['xmax']/df['width']
df['YMin'] = df['ymin']/df['height']
df['YMax'] = df['ymax']/df['height']

df.drop(columns={'filename','xmin','xmax','ymin','ymax'},inplace=True)
df = df.rename(columns={'class':'LabelName'})
df = df[['ImageID','LabelName','XMin','XMax','YMin','YMax' ]]
df

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,bus,0.242788,0.418269,0.360577,0.504808
1,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.014423,0.052885,0.411058,0.519231
2,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,three wheelers (CNG),0.218750,0.250000,0.430288,0.519231
3,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.391827,0.427885,0.432692,0.526442
4,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.250000,0.290865,0.413462,0.545673
...,...,...,...,...,...,...
284,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,three wheelers (CNG),0.766827,0.822115,0.399038,0.430288
285,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,rickshaw,0.848558,0.882212,0.401442,0.432692
286,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,rickshaw,0.319712,0.430288,0.377404,0.480769
287,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,three wheelers (CNG),0.002404,0.074519,0.365385,0.509615


In [425]:
# checking
df["ImageID"].nunique()

30

In [426]:
if roll =='005' :
  orgdf = pd.DataFrame()
else :
  orgdf = pd.read_csv('/content/drive/MyDrive/Dhaka-AI 2020/Handlabel/handlabel.csv')
print(len(orgdf))
pdList = [orgdf,df]
new_df = pd.concat(pdList)
print(len(new_df))

832
1121


In [427]:
new_df['LabelName'].unique()

# 'bus' 'rickshaw' 'motorbike' 'car' 'three wheelers (CNG)' 'pickup'
#  'minivan' 'suv' 'van' 'taxi' 'truck' 'bicycle' 'policecar' 'ambulance'
#  'human hauler' 'wheelbarrow' 'minibus' 'auto rickshaw' 'army vehicle'
#  'scooter' 'garbagevan'

array(['car', 'bus', 'van', 'minibus', 'rickshaw', 'motorbike',
       'three wheelers (CNG)', 'suv', 'pickup', 'minivan', 'police car',
       'bicycle', 'truck', 'human hauler', 'wheelbarrow', 'auto rickshaw',
       'policecar', 'ambulance'], dtype=object)

In [428]:
new_df['LabelName'] = new_df['LabelName'].replace(['Three Wheelers (CNG)'],'three wheelers (CNG)')
new_df['LabelName'] = new_df['LabelName'].replace(['three wheelers(CNG)'],'three wheelers (CNG)')
new_df['LabelName'] = new_df['LabelName'].replace(['Truck'],'truck')
new_df['LabelName'] = new_df['LabelName'].replace(['buss'],'bus')
print(len(new_df['LabelName'].unique()))
# new_df['ImageID'] = new_df['ImageID'].str.replace('sraf', 'Asraf')
new_df.to_csv('/content/drive/MyDrive/Dhaka-AI 2020/Handlabel/handlabel.csv',index=False)

18


In [429]:
new_df[new_df['ImageID'].str.contains("sraf")]

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,bus,0.242788,0.418269,0.360577,0.504808
1,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.014423,0.052885,0.411058,0.519231
2,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,three wheelers (CNG),0.218750,0.250000,0.430288,0.519231
3,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.391827,0.427885,0.432692,0.526442
4,Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c,rickshaw,0.250000,0.290865,0.413462,0.545673
...,...,...,...,...,...,...
257,Asraf_97_jpg.rf.99ed4d371ef0d45df178a79a031f260e,three wheelers (CNG),0.420673,0.588942,0.387019,0.673077
258,Asraf_97_jpg.rf.99ed4d371ef0d45df178a79a031f260e,bicycle,0.935096,0.992788,0.495192,0.677885
259,Asraf_97_jpg.rf.99ed4d371ef0d45df178a79a031f260e,rickshaw,0.778846,0.918269,0.326923,0.932692
260,Asraf_97_jpg.rf.99ed4d371ef0d45df178a79a031f260e,motorbike,0.151442,0.348558,0.555288,0.870192


In [430]:
new_df

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,Shykat_5_ (84)_jpg.rf.490f1b051d6cbf286cce8832...,car,0.334135,0.353365,0.449519,0.485577
1,Shykat_5_ (84)_jpg.rf.490f1b051d6cbf286cce8832...,car,0.288462,0.317308,0.463942,0.490385
2,Shykat_5_ (84)_jpg.rf.490f1b051d6cbf286cce8832...,bus,0.423077,0.487981,0.430288,0.531250
3,Shykat_5_ (84)_jpg.rf.490f1b051d6cbf286cce8832...,van,0.514423,0.552885,0.480769,0.533654
4,Shykat_5_ (84)_jpg.rf.490f1b051d6cbf286cce8832...,minibus,0.360577,0.403846,0.451923,0.574519
...,...,...,...,...,...,...
284,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,three wheelers (CNG),0.766827,0.822115,0.399038,0.430288
285,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,rickshaw,0.848558,0.882212,0.401442,0.432692
286,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,rickshaw,0.319712,0.430288,0.377404,0.480769
287,Pias (581)_jpg.rf.c6dd15178885b22161c006edcf52...,three wheelers (CNG),0.002404,0.074519,0.365385,0.509615
